In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder

train=train.drop(['EmployeeNumber','StandardHours'],axis=1)
test = test.drop(['EmployeeNumber','StandardHours'],axis=1)

attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.fit_transform(test[feature])
    lbe_list.append(lbe)
import xgboost as xgb
from sklearn.model_selection import train_test_split
param={'boosting_type':'gbdt',
      'objective':'binary:logistic',
      'eval_metric':'auc',
      'eta':0.01,
      'max_depth':15,
      'colsample_bytree':0.8,
      'subsample':0.9,
      "subsample_freq":8,
      'alpha':0.6,
      'lambda':0,
      }
X_train,X_valid,y_train,y_valid=train_test_split(train.drop(['Attrition'],axis=1),train['Attrition'],test_size=0.2,random_state=42)

train_data=xgb.DMatrix(X_train,label=y_train)
valid_data=xgb.DMatrix(X_valid,label=y_valid)
test_data=xgb.DMatrix(test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)
test['Attrition']=predict
# 转化为二分类输出
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
test[['user_id','Attrition']].to_csv('submit_xgb.csv')

[10:55:57] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { boosting_type, subsample_freq } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.81897	valid-auc:0.64018
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.
[25]	train-auc:0.96636	valid-auc:0.76596
[50]	train-auc:0.97783	valid-auc:0.77387
[75]	train-auc:0.98741	valid-auc:0.77966
[100]	train-auc:0.99198	valid-auc:0.77944
[125]	train-auc:0.99409	valid-auc:0.77791
[150]	train-auc:0.99654	valid-auc:0.78348
[175]	train-auc:0.99787	valid-auc:0.78577
[200]	train-auc:0.99872	valid-auc:0.79101
[225]	train-auc:0.99920	valid-auc:0.79472
[250]	train-auc:0.99960	valid-auc: